# Coletando os Dados

In [ ]:
# encoding: utf-8
from __future__ import unicode_literals
import requests
from lxml import html

feam_url = "http://www.feam.br/noticias/1/1327-boletim-qualidade-do-ar"
page_content = requests.get(feam_url).content
html_tree = html.fromstring(page_content)
html_tree.resolve_base_href()
elements = html_tree.xpath('//a')


In [2]:
possible_matches = []
for element in elements:
    link = element.get('href')
    if link is not None and ".xlsx" in link:
        print link
        possible_matches.append(link)

        
        

http://www.feam.br/images/Boletim_-_Novembro.xlsx
http://www.feam.br/images/Boletim_-_Novembro.xlsx
http://www.feam.br/images/Boletim_Maio.xlsx
http://www.feam.br/images/Boletim_Maio.xlsx


In [3]:
from datetime import datetime
months = ['', 'Janeiro', 'Fevereiro', 'Março', 'Abril', 'Maio', 'Junho', 'Julho', 'Agosto', 'Setembro', 'Outubro', 'Novembro', 'Dezembro']
date = datetime.now()

In [4]:
months[date.month]

u'Maio'

In [5]:
downloads = 0
for possible_match in possible_matches:
    if months[date.month] in possible_match:
        file_name = 'Dados/Dados-' + str(date.day) + '-' + str(date.month) + '-' + str(date.year) + '-' + str(downloads) + '.xlsx'
        result = requests.get(possible_match)
        downloads = downloads + 1
        with open(file_name, 'wb') as f:
            print(file_name)
            f.write(result.content)
        

Dados/Dados-8-5-2018-0.xlsx
Dados/Dados-8-5-2018-1.xlsx


# Ingestão dos dados de Poluição atmosférica

In [6]:
class Polutant(object):
    def __init__(self, name, quantity, date=None):
        self.name = name
        self.quantity = quantity
        self.date = date
     

In [7]:
import csv
from datetime import datetime
report_by_day = {}
first_row = True
date = None
with open('Dados/Poluentes.csv', 'rb') as f:
    reader = csv.reader(f)
    for row in reader:
        if first_row: # Skipping header row
            first_row = False
        else:
            first_col = True
            list_polutants = []
            qtd_polutants = 0
            for col in row:
                if first_col:
                    date = datetime.strptime(col, '%d-%m-%y') # first value is always the date
                    first_col = False
                elif col:
                    if not col.isdigit(): # if it's only numbers it's a polutants quantity
                        list_polutants.append(Polutant(col.strip(), 0))
                    else:
                        p = list_polutants[qtd_polutants]
                        p.quantity = int(col)
                        p.date = date
                        list_polutants[qtd_polutants] = p
                        qtd_polutants = qtd_polutants + 1
                        
           # report_by_day[date] = dict([(p.name, p.quantity) for p in list_polutants])
            report_by_day[date] = list_polutants
        
                    
                

In [8]:
for date in sorted(report_by_day):
    print date.strftime('%d-%m-%y')
    for p in report_by_day.get(date):
        print p.name
        print p.quantity
        print p.date.strftime('%d-%m-%y')
    print '--------------'


27-12-16
--------------
28-12-16
--------------
29-12-16
--------------
30-12-16
--------------
31-12-16
--------------
01-01-17
--------------
02-01-17
--------------
03-01-17
--------------
04-01-17
--------------
05-01-17
--------------
06-01-17
--------------
07-01-17
--------------
08-01-17
--------------
09-01-17
--------------
10-01-17
--------------
11-01-17
--------------
12-01-17
--------------
13-01-17
--------------
14-01-17
--------------
15-01-17
--------------
16-01-17
--------------
17-01-17
--------------
18-01-17
--------------
19-01-17
--------------
20-01-17
--------------
21-01-17
--------------
22-01-17
--------------
23-01-17
--------------
24-01-17
--------------
25-01-17
--------------
26-01-17
--------------
27-01-17
--------------
28-01-17
--------------
29-01-17
--------------
30-01-17
--------------
31-01-17
--------------
01-02-17
--------------
02-02-17
--------------
03-02-17
--------------
04-02-17
--------------
05-02-17
--------------
06-02-17
-------

# Ingestão dos dados epidemiológicos

In [9]:
import unicodecsv

with open('Dados/DoencasRespiratorias.csv', 'rb') as f:
    reader = unicodecsv.DictReader(f)
    diseases = list(reader)

In [10]:
diseases

[{u'A-caso': u'0',
  u'A-obito': u'0',
  u'Ano': u'2017',
  u'B-caso': u'1',
  u'B-obito': u'1',
  u'H1N1-caso': u'0',
  u'H1N1-obito': u'0',
  u'H3-caso': u'2',
  u'H3-obito': u'0',
  u'SRAG-caso': u'15',
  u'SRAG-obito': u'1',
  u'Semana': u'1'},
 {u'A-caso': u'0',
  u'A-obito': u'0',
  u'Ano': u'2017',
  u'B-caso': u'1',
  u'B-obito': u'1',
  u'H1N1-caso': u'0',
  u'H1N1-obito': u'0',
  u'H3-caso': u'2',
  u'H3-obito': u'0',
  u'SRAG-caso': u'25',
  u'SRAG-obito': u'6',
  u'Semana': u'2'},
 {u'A-caso': u'0',
  u'A-obito': u'0',
  u'Ano': u'2017',
  u'B-caso': u'1',
  u'B-obito': u'0',
  u'H1N1-caso': u'0',
  u'H1N1-obito': u'0',
  u'H3-caso': u'0',
  u'H3-obito': u'0',
  u'SRAG-caso': u'17',
  u'SRAG-obito': u'0',
  u'Semana': u'3'},
 {u'A-caso': u'0',
  u'A-obito': u'0',
  u'Ano': u'2017',
  u'B-caso': u'1',
  u'B-obito': u'0',
  u'H1N1-caso': u'0',
  u'H1N1-obito': u'0',
  u'H3-caso': u'2',
  u'H3-obito': u'0',
  u'SRAG-caso': u'11',
  u'SRAG-obito': u'2',
  u'Semana': u'4'},
 {u'

# Exploração de dados de poluição

In [11]:
max_polutants = []
for date in sorted(report_by_day):
    for p in report_by_day.get(date):
        if not any(x.name == p.name for x in max_polutants):
            max_polutants.append(p)
        else:
            for x in max_polutants:
                if x.name == p.name:
                    if x.quantity < p.quantity:
                        max_polutants.remove(x)
                        max_polutants.append(p)
                    else:
                        break
for p in max_polutants:
    print p.name
    print p.quantity
    print p.date.strftime('%d-%m-%y')
    print '--------------'        
    

Dioxido de Nitrogenio
77
23-03-17
--------------
Monoxido de Carbono
11
25-05-17
--------------
Ozonio
111
10-08-17
--------------
Particulas Totais em Suspensao
34
25-09-17
--------------
Particulas Inalaveis
84
12-03-18
--------------


# Exploração de dados epidemiológicos

In [12]:
import sys
lo,hi = sys.maxint,-sys.maxint-1
dateLo, dateHi = None, None

for disease in diseases:
    if int(disease['A-caso'])<lo:
        lo=int(disease['A-caso'])
        dateLo=disease['Semana']+"-"+disease['Ano']
    if int(disease['A-caso'])>hi:
        hi=int(disease['A-caso'])
        dateHi=disease['Semana']+"-"+disease['Ano']
        
print "Valor máximo e data de casos de Influenza A:", hi, ",", dateHi
print "Valor mínimo e data de casos de Influenza A:", lo, ",", dateLo


Valor máximo e data de casos de Influenza A: 3 , 18-2017
Valor mínimo e data de casos de Influenza A: 0 , 1-2017


In [13]:
import sys
lo,hi = sys.maxint,-sys.maxint-1
dateLo, dateHi = None, None

for disease in diseases:
    if int(disease['A-obito'])<lo:
        lo=int(disease['A-obito'])
        dateLo=disease['Semana']+"-"+disease['Ano']
    if int(disease['A-obito'])>hi:
        hi=int(disease['A-obito'])
        dateHi=disease['Semana']+"-"+disease['Ano']
        
print "Valor máximo e data de óbitos por Influenza A:", hi, ",", dateHi
print "Valor mínimo e data de óbitos por Influenza A:", lo, ",", dateLo


Valor máximo e data de óbitos por Influenza A: 2 , 18-2017
Valor mínimo e data de óbitos por Influenza A: 0 , 1-2017


In [14]:
import sys
lo,hi = sys.maxint,-sys.maxint-1
dateLo, dateHi = None, None

for disease in diseases:
    if int(disease['B-caso'])<lo:
        lo=int(disease['B-caso'])
        dateLo=disease['Semana']+"-"+disease['Ano']
    if int(disease['B-caso'])>hi:
        hi=int(disease['B-caso'])
        dateHi=disease['Semana']+"-"+disease['Ano']
        
print "Valor máximo e data de casos de Influenza B:", hi, ",", dateHi
print "Valor mínimo e data de casos de Influenza B:", lo, ",", dateLo


Valor máximo e data de casos de Influenza B: 2 , 20-2017
Valor mínimo e data de casos de Influenza B: 0 , 5-2017


In [15]:
import sys
lo,hi = sys.maxint,-sys.maxint-1
dateLo, dateHi = None, None

for disease in diseases:
    if int(disease['B-obito'])<lo:
        lo=int(disease['B-obito'])
        dateLo=disease['Semana']+"-"+disease['Ano']
    if int(disease['B-obito'])>hi:
        hi=int(disease['B-obito'])
        dateHi=disease['Semana']+"-"+disease['Ano']
        
print "Valor máximo e data de óbitos por Influenza B:", hi, ",", dateHi
print "Valor mínimo e data de óbitos por Influenza B:", lo, ",", dateLo

Valor máximo e data de óbitos por Influenza B: 1 , 1-2017
Valor mínimo e data de óbitos por Influenza B: 0 , 3-2017


In [16]:
import sys
lo,hi = sys.maxint,-sys.maxint-1
dateLo, dateHi = None, None

for disease in diseases:
    if int(disease['H1N1-caso'])<lo:
        lo=int(disease['H1N1-caso'])
        dateLo=disease['Semana']+"-"+disease['Ano']
    if int(disease['H1N1-caso'])>hi:
        hi=int(disease['H1N1-caso'])
        dateHi=disease['Semana']+"-"+disease['Ano']
        
print "Valor máximo e data de casos de Influenza H1N1:", hi, ",", dateHi
print "Valor mínimo e data de casos de Influenza H1N1:", lo, ",", dateLo


Valor máximo e data de casos de Influenza H1N1: 3 , 14-2017
Valor mínimo e data de casos de Influenza H1N1: 0 , 1-2017


In [17]:
import sys
lo,hi = sys.maxint,-sys.maxint-1
dateLo, dateHi = None, None

for disease in diseases:
    if int(disease['H1N1-obito'])<lo:
        lo=int(disease['H1N1-obito'])
        dateLo=disease['Semana']+"-"+disease['Ano']
    if int(disease['H1N1-obito'])>hi:
        hi=int(disease['H1N1-obito'])
        dateHi=disease['Semana']+"-"+disease['Ano']
        
print "Valor máximo e data de óbitos por Influenza H1N1:", hi, ",", dateHi
print "Valor mínimo e data de óbitos por Influenza H1N1:", lo, ",", dateLo

Valor máximo e data de óbitos por Influenza H1N1: 0 , 1-2017
Valor mínimo e data de óbitos por Influenza H1N1: 0 , 1-2017


In [18]:
import sys
lo,hi = sys.maxint,-sys.maxint-1
dateLo, dateHi = None, None

for disease in diseases:
    if int(disease['H3-caso'])<lo:
        lo=int(disease['H3-caso'])
        dateLo=disease['Semana']+"-"+disease['Ano']
    if int(disease['H3-caso'])>hi:
        hi=int(disease['H3-caso'])
        dateHi=disease['Semana']+"-"+disease['Ano']
        
print "Valor máximo e data de casos de Influenza H3:", hi, ",", dateHi
print "Valor mínimo e data de casos de Influenza H3:", lo, ",", dateLo

Valor máximo e data de casos de Influenza H3: 21 , 21-2017
Valor mínimo e data de casos de Influenza H3: 0 , 3-2017


In [19]:
import sys
lo,hi = sys.maxint,-sys.maxint-1
dateLo, dateHi = None, None

for disease in diseases:
    if int(disease['H3-obito'])<lo:
        lo=int(disease['H3-obito'])
        dateLo=disease['Semana']+"-"+disease['Ano']
    if int(disease['H3-obito'])>hi:
        hi=int(disease['H3-obito'])
        dateHi=disease['Semana']+"-"+disease['Ano']
        
print "Valor máximo e data de óbitos por Influenza H3:", hi, ",", dateHi
print "Valor mínimo e data de óbitos por Influenza H3:", lo, ",", dateLo

Valor máximo e data de óbitos por Influenza H3: 4 , 20-2017
Valor mínimo e data de óbitos por Influenza H3: 0 , 1-2017


In [20]:
import sys
lo,hi = sys.maxint,-sys.maxint-1
dateLo, dateHi = None, None

for disease in diseases:
    if int(disease['SRAG-caso'])<lo:
        lo=int(disease['SRAG-caso'])
        dateLo=disease['Semana']+"-"+disease['Ano']
    if int(disease['SRAG-caso'])>hi:
        hi=int(disease['SRAG-caso'])
        dateHi=disease['Semana']+"-"+disease['Ano']
        
print "Valor máximo e data de casos de SRAG:", hi, ",", dateHi
print "Valor mínimo e data de casos de SRAG:", lo, ",", dateLo

Valor máximo e data de casos de SRAG: 57 , 20-2017
Valor mínimo e data de casos de SRAG: 0 , 28-2017


In [21]:
import sys
lo,hi = sys.maxint,-sys.maxint-1
dateLo, dateHi = None, None

for disease in diseases:
    if int(disease['SRAG-obito'])<lo:
        lo=int(disease['SRAG-obito'])
        dateLo=disease['Semana']+"-"+disease['Ano']
    if int(disease['SRAG-obito'])>hi:
        hi=int(disease['SRAG-obito'])
        dateHi=disease['Semana']+"-"+disease['Ano']
        
print "Valor máximo e data de óbitos por Influenza SRAG:", hi, ",", dateHi
print "Valor mínimo e data de óbitos por Influenza SRAG:", lo, ",", dateLo

Valor máximo e data de óbitos por Influenza SRAG: 9 , 12-2017
Valor mínimo e data de óbitos por Influenza SRAG: 0 , 3-2017


# Aplicação do método k-means aos dados

In [22]:
import pandas as pd
df_diseases = pd.DataFrame(diseases)

In [23]:
df_diseases

,A-caso,A-obito,Ano,B-caso,B-obito,H1N1-caso,H1N1-obito,H3-caso,H3-obito,SRAG-caso,SRAG-obito,Semana
0,0,0,2017,1,1,0,0,2,0,15,1,1
1,0,0,2017,1,1,0,0,2,0,25,6,2
2,0,0,2017,1,0,0,0,0,0,17,0,3
3,0,0,2017,1,0,0,0,2,0,11,2,4
4,1,0,2017,0,0,0,0,1,0,15,0,5
5,0,0,2017,0,0,0,0,2,1,25,2,6
6,0,0,2017,1,0,0,0,4,1,23,4,7
7,0,0,2017,0,0,0,0,2,0,17,2,8
8,0,0,2017,1,0,0,0,3,0,23,3,9
9,1,0,2017,0,0,0,0,3,0,18,0,10


In [24]:
index = list(sorted(report_by_day.keys()))

columns = ['NO2', 'CO', 'O3', 'PTS', 'PI']
df_polutants = pd.DataFrame(index=index, columns=columns)
df_polutants = df_polutants.fillna(0) # with 0s rather than NaNs

In [25]:
df_polutants.head(10)

,NO2,CO,O3,PTS,PI
2016-12-27,0,0,0,0,0
2016-12-28,0,0,0,0,0
2016-12-29,0,0,0,0,0
2016-12-30,0,0,0,0,0
2016-12-31,0,0,0,0,0
2017-01-01,0,0,0,0,0
2017-01-02,0,0,0,0,0
2017-01-03,0,0,0,0,0
2017-01-04,0,0,0,0,0
2017-01-05,0,0,0,0,0


In [26]:
count = 0
for date in list(sorted(report_by_day.keys())):
    for p in report_by_day.get(date):
        if p.name == 'Dioxido de Nitrogenio':
            df_polutants.loc[date,'NO2'] = p.quantity
        elif p.name == 'Monoxido de Carbono':
            df_polutants.loc[date,'CO'] = p.quantity
        elif p.name == 'Ozonio':
            df_polutants.loc[date,'O3'] = p.quantity
        elif p.name == 'Particulas Totais em Suspensao':
            df_polutants.loc[date,'PTS'] = p.quantity
        elif p.name == 'Particulas Inalaveis':
            df_polutants.loc[date,'PI'] = p.quantity

In [27]:
df_polutants.head(200)

,NO2,CO,O3,PTS,PI
2016-12-27,0,0,0,0,0
2016-12-28,0,0,0,0,0
2016-12-29,0,0,0,0,0
2016-12-30,0,0,0,0,0
2016-12-31,0,0,0,0,0
2017-01-01,0,0,0,0,0
2017-01-02,0,0,0,0,0
2017-01-03,0,0,0,0,0
2017-01-04,0,0,0,0,0
2017-01-05,0,0,0,0,0


In [28]:
index2 = [(df_diseases['Semana']),(df_diseases['Ano'])]
columns = ['NO2', 'CO', 'O3', 'PTS', 'PI']
df_polutants_by_week_max = pd.DataFrame(index=index2, columns=columns)
df_polutants_by_week_max = df_polutants_by_week_max.fillna(0) # with 0s rather than NaNs
df_polutants_by_week_mean = pd.DataFrame(index=index2, columns=columns)
df_polutants_by_week_mean = df_polutants_by_week_mean.fillna(0) # with 0s rather than NaNs
df_polutants_by_week_sum = pd.DataFrame(index=index2, columns=columns)
df_polutants_by_week_sum = df_polutants_by_week_sum.fillna(0) # with 0s rather than NaNs

In [30]:
df_polutants_by_week_mean


,,NO2,CO,O3,PTS,PI
Semana,Ano,,,,,
1,2017,0,0,0,0,0
2,2017,0,0,0,0,0
3,2017,0,0,0,0,0
4,2017,0,0,0,0,0
5,2017,0,0,0,0,0
6,2017,0,0,0,0,0
7,2017,0,0,0,0,0
8,2017,0,0,0,0,0
9,2017,0,0,0,0,0
